Внимание! 18+!  
Соревнование на kaggle классификация токсичных комментариев.

https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/overview

Датасет содержит комментарии на викепедии и ручную размеку принадлежности комментария к категориям:  
1. toxic
2. severe toxic
3. obscene
4. threat
5. insult
6. identity hate

Один коментарий может принадлежать к нескольким категориям, а может не принадлежать ни к одной (не токсичный комментарий).  
Важное замечание, что для валидации, необходимо определить вероятнось принадлежности к классу.

In [1]:
import pandas as pd
import numpy as np

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data=pd.read_csv('train.csv')

In [3]:
data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
data.comment_text[123]

'Should say something about his views as an educationalist and socialist political commentator.\n\nLink to http://www.langandlit.ualberta.ca/Fall2004/SteigelBainbridge.html mentions this a bit - he stood as an election candidate for Respect.'

In [5]:
data.comment_text[1]

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [6]:
data.comment_text[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

Данные зашумлены. Имеются символы перевода строки, ссылки, дата комментария, ip адреса.

Необходима функция очистки данных и лемматизатор.

In [23]:
import nltk
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from string import punctuation
from sklearn.preprocessing import FunctionTransformer
from nltk.corpus import wordnet

stem = nltk.stem.WordNetLemmatizer() 
stopwords = stopwords.words("english")
punctuation=punctuation+'0123456789 '

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def preprocess_text(X):
    prep_text=[]
    for text in X:
        text = text.replace('\n',' ')
        tokens = [token.strip(punctuation).strip() for token in text.lower().split() if token not in stopwords\
                  and token.strip(punctuation) != "" \
                  and token != " " \
                  and token != "" \
                  and token.strip() not in punctuation]
        
        tokens = [stem.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
        text = " ".join(tokens)
    
        prep_text.append(text)
    return prep_text

[nltk_data] Downloading package stopwords to /home/kirill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kirill/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [24]:
data.comment_text[20]

'"\n\n Regarding your recent edits \n\nOnce again, please read WP:FILMPLOT before editing any more film articles.  Your edits are simply not good, with entirely too many unnecessary details and very bad writing.  Please stop before you do further damage. -\'\'\'\'\'\'The \'45 "'

In [25]:
preprocess_text([data.comment_text[20],])

['regard recent edits again please read wp:filmplot edit film article edits simply good entirely many unnecessary detail bad write please stop damage the']

Подготавливаю данные.

In [26]:
%%time
X=preprocess_text(data.comment_text)

CPU times: user 22min 54s, sys: 53.4 s, total: 23min 48s
Wall time: 23min 49s


Применяю векторизатор TfidfVectorizer

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [32]:
classes=['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [33]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X=vectorizer.fit_transform(X)

Я буду обучать и строить прогнозы для каждого класса отдельно.

Загружаю тестовые данные.

In [35]:
test=pd.read_csv('test.csv')

In [36]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


Очищаю и векторизую тестовые данные.

In [37]:
X_test=preprocess_text(test.comment_text)

In [38]:
X_test=vectorizer.transform(X_test)

Обучаю и строю прогноз по каждому классу

In [45]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in classes:
    y = data[class_name]
    classifier = LogisticRegression(random_state=123)

    cv_score = np.mean(cross_val_score(classifier, X, y, cv=5, scoring='roc_auc', n_jobs=2))
    scores.append(cv_score)
    print('CV score для класса {} - {}'.format(class_name, cv_score))

    classifier.fit(X, y)
    submission[class_name] = classifier.predict_proba(X_test)[:, 1]

print('Итоговый CV score - {}'.format(np.mean(scores)))

submission.to_csv('submission.csv', index=False)

CV score для класса toxic - 0.967538664725925
CV score для класса severe_toxic - 0.9861703839047795
CV score для класса obscene - 0.9841955124101502
CV score для класса threat - 0.9784328345624737
CV score для класса insult - 0.9755872480772657
CV score для класса identity_hate - 0.9736272959458576
Итоговый CV score - 0.9775919899377419


Отличный результат. Посмотрю на ответы.

In [46]:
submission.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.996057,0.161723,0.992190,0.017263,0.964460,0.195967
1,0000247867823ef7,0.032441,0.005039,0.018053,0.002494,0.021913,0.006506
2,00013b17ad220c46,0.037636,0.004809,0.017440,0.002156,0.018382,0.004885
3,00017563c3f7919a,0.006496,0.002870,0.005622,0.001645,0.005772,0.002443
4,00017695ad8997eb,0.027548,0.002995,0.010447,0.001567,0.011833,0.002493
5,0001ea8717f6de06,0.012470,0.002568,0.007065,0.001737,0.009754,0.003027
6,00024115d4cbde0f,0.007245,0.001361,0.004847,0.001195,0.005269,0.002025
7,000247e83dcc1211,0.336221,0.007006,0.055176,0.003558,0.082013,0.009918
8,00025358d4737918,0.036860,0.003561,0.014369,0.002362,0.017393,0.004937
9,00026d1092fe71cc,0.014838,0.002823,0.008748,0.001801,0.009936,0.003659


In [47]:
test.head(10)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


На Kagle получилась точность 97.47%